# ReAct

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_experimental.tools import PythonREPLTool

In [3]:
llm = ChatOpenAI(model="gpt-4-0613")

tools = [
    DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region="de-de")),
    PythonREPLTool()
]

agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_execution_time=120
)

In [4]:
agent_chain.invoke("Wann beginnt der Christkindlmarkt auf dem Neupfarrplatz 2023? Wie viele Tage sind es bis dahin?")



> Entering new AgentExecutor chain...
Thought: This question is asking two things: when the Christkindlmarkt (Christmas Market) begins on the Neupfarrplatz in 2023, and how many days are left until the start of the market. To answer this, I'll first need to find out the start date of the market in 2023. I will use the DuckDuckGo search tool to find this information. Then, I'll calculate the difference between the current date and the start date of the market using a Python command. 

Action:
```
{
  "action": "duckduckgo_search",
  "action_input": "Start date of Christkindlmarkt Neupfarrplatz 2023"
}
```
Observation: Regensburg Christmas Market dates: 27th November to 23rd December 2023. Location: Neupfarrplatz. Opening times: Sunday to Wednesday: 10am to 8pm. Thursday to Saturday: 10am to 9pm. Christmas Day: Closed. Admission: Free. Christmas lights: To save energy, the church will not be illuminated. Santa: Yes, every Wednesday is children's day. am 18.09.2023 aktualisiert Der Glüh

Python REPL can execute arbitrary code. Use with caution.


The start date of the Christkindlmarkt at Neupfarrplatz in 2023 is November 27. Now I need to calculate the number of days between today's date and November 27, 2023. For that, I will use the Python_REPL tool to execute the necessary Python commands.

Action:
```
{
  "action": "Python_REPL",
  "action_input": "from datetime import datetime\ndays_left = (datetime(2023, 11, 27) - datetime.now()).days\nprint(days_left)"
}
```
Observation: 24

Thought:The Christkindlmarkt at Neupfarrplatz begins on November 27, 2023, and there are 24 days left until it starts.

Final Answer: Der Christkindlmarkt auf dem Neupfarrplatz beginnt am 27. November 2023. Es sind noch 24 Tage bis dahin.

> Finished chain.


{'input': 'Wann beginnt der Christkindlmarkt auf dem Neupfarrplatz 2023? Wie viele Tage sind es bis dahin?',
 'output': 'Der Christkindlmarkt auf dem Neupfarrplatz beginnt am 27. November 2023. Es sind noch 24 Tage bis dahin.'}